# Fix simulation sier results
7 genes' sier PIP are greater than 1 since the summation of whole genome varbvs PIPs are greater than 1.

In [1]:
import pandas as pd, numpy as np
import os
from collections import Counter
cwd = os.path.expanduser("/home/min/GIT/cnv-gene-mapping/output")
name = "simu_dele_30_0528"
prefix = f"deletion.X.colnames_b30.{name}"

In [2]:
index = pd.read_csv(f"{cwd}/{name}/{prefix}.X.block_index.csv", sep = "\t", header = None, names = ["start", "end"])

In [3]:
index.shape

(565, 2)

For some big blocks, sier consumes more than 3 hours, resulting in no outcome in [sier_3], so there is no overall outcome for all blocks. Use MCMC result instead temporarily and combine sier results in all blocks with workflow command.

In [4]:
for i, item in index.iterrows():
    sumpip = np.sum(np.loadtxt(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.varbvs_pip", usecols = 1))
    if sumpip > 1:
        sier = pd.read_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.sier.pip", sep = "\t", header = None, names = ["gene", "pip"])
        sier["pip"] = [x/sumpip for x in sier["pip"]]
#         sier.to_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.sier.pip", sep = "\t", header = False, index = False)

Fit hybrid outcome: combine MCMC and sier results after determining blocked whole genome varbvs summation. If summation is greater than or equal to 1, use MCMC result, otherwise, use sier result.

In [5]:
res = pd.DataFrame(columns = ["gene", "pip"])
j = 0
for i, item in index.iterrows():
    sumpip = np.sum(np.loadtxt(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.varbvs_pip", usecols = 1))
    if sumpip >= 1:
        pips = pd.read_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.mcmc.pip", sep = "\t", header = None, names = ["gene", "pip"])
        j += 1
    else:
        pips = pd.read_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.sier.pip", sep = "\t", header = None, names = ["gene", "pip"])
    res = pd.concat([res, pips])
res.to_csv(f"{cwd}/{name}/{prefix}.y.hybrid_pip.gz", compression = "gzip", sep = "\t", header = False, index = False)

In [6]:
res.shape

(2756, 2)

In [7]:
res.head()

,gene,pip
0,TTLL10,0.012965
1,TNFRSF18,0.012965
2,TNFRSF4,0.012965
3,SDF4,0.012965
4,B3GALT6,0.012965


In [8]:
j

49